In [3]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer

2022-02-21 13:36:46,641 dwave.cloud INFO MainThread Log level for 'dwave.cloud' namespace set to 0


# Step 1: Prepare Data

In [4]:
# initial parameters for experiment data
s3_bucket = f"amazon-braket-1a222675c751" # the name of the bucket
prefix = "annealer-experiment" # the name of the folder in the bucket

raw_path = './molecule-data/117_ideal.mol2' # the mol2 file for this experiment

In [5]:
mol_data = MoleculeData(raw_path, 'qmu')

data_path = mol_data.save("latest")

num_rotation_bond = mol_data.bond_graph.rb_num
print(f"You have loaded the raw molecule data and saved as {data_path}. \n\
This molecule has {num_rotation_bond} rotable bond")

INFO:root:parse mol2 file!
INFO:root:finish save qmu_117_ideal_data_latest.pickle


You have loaded the raw molecule data and saved as ./qmu_117_ideal_data_latest.pickle. 
This molecule has 23 rotable bond


# Step 2: Build Model

In [27]:
# initial the QMUQUBO object
init_param = {}
method = ['pre-calc']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

INFO:root:initial pre-calculate for constructing molecule QUBO


In [28]:
# set the parameters for model
model_param = {}
# parameters
num_rotation_bond = mol_data.bond_graph.rb_num

method = 'pre-calc'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [4]
model_param[method]['D'] = [4]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]

qmu_qubo.build_model(**model_param)

INFO:root:Construct model for M:4,D:4,A:300,hubo_qubo_val:200 0.006114335854848226 min


0

In [29]:
# describe the model parameters
model_info = qmu_qubo.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {4}
INFO:root:param: D, value {4}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [30]:
# save the model
model_path = qmu_qubo.save("latest")

print(f"You have built the QUBO model and saved it as {model_path}")

INFO:root:finish save qmu_117_ideal_model_latest.pickle


You have built the QUBO model and saved it as ./qmu_117_ideal_model_latest.pickle


# Step 3: Optimize Configuration

In [31]:
qmu_qubo_optimize = QMUQUBO.load(model_path)

In [32]:
model_info = qmu_qubo_optimize.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {4}
INFO:root:param: D, value {4}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [33]:
# get the model you want to optimize
M = 4
D = 4
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

In [34]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 1

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use simulated annealer from dimod


In [35]:
sa_optimize_result = sa_optimizer.fit()

INFO:root:fit() ...
INFO:root:dwave-sa save to local
INFO:root:finish save sa_result.pickle


In [36]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:root:use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


In [37]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

INFO:root:fit() ...
INFO:root:finish save /tmp/qa_result.pickle
INFO:root:_upload_result_json, bucket=amazon-braket-1a222675c751, key=annealer-experiment/79a63d33-ffb1-496f-b824-68bc2f1352c6/qa_result.pickle
INFO:root:dwave-qa save to s3 - 79a63d33-ffb1-496f-b824-68bc2f1352c6: None


In [38]:
qa_task_id = qa_optimizer.get_task_id()
print(f"task id is {qa_task_id}")

task id is 79a63d33-ffb1-496f-b824-68bc2f1352c6


In [39]:
print(f"dwave-sa run time {sa_optimize_result['time']}")
print(f"dwave-qa run time {qa_optimize_result['time']}")

dwave-sa run time 0.683696985244751
dwave-qa run time 25.150969743728638


# Step 4: PostProcess Result

In [1]:
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [2]:
qa_task_id = '79a63d33-ffb1-496f-b824-68bc2f1352c6'
s3_bucket = f"amazon-braket-1a222675c751" # the name of the bucket
prefix = "annealer-experiment" # the name of the folder in the bucket
data_path = './qmu_117_ideal_data_latest.pickle'
raw_path = './molecule-data/117_ideal.mol2' # the mol2 file for this experiment

In [3]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 0.683696985244751


In [4]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_process_result.save_mol_file(f"{timestamp}")


INFO:root:generate_optimize_pts()
DEBUG:root:generate_optimize_pts model_info={'model_name': '4_4_300_200', 'rb_name': ['30+31', '29+30', '28+29', '33+34'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4

['./molecule-data/117_ideal_dwave-sa_20220221-13.mol2',
 './molecule-data/117_ideal_dwave-sa_20220221-13.json']

In [5]:
sa_process_result.parameters

{'volume': {'optimize': 65.47551386810179,
  'initial': 65.31411358299036,
  'gain': 1.0024711394866646,
  'unfolding_results': ['x_15_4', 'x_14_4', 'x_12_4', 'x_18_2']}}

In [6]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/79a63d33-ffb1-496f-b824-68bc2f1352c6/qa_result.pickle
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function sse_md5 at 0x7fd1960b4dd0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function validate_bucket_name at 0x7fd1960b4d40>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <bound method S3RegionRedirector.redirect_from_cache of <botocore.utils.S3RegionRedirector object at 0x7fd198492d50>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <bound method S3ArnParamHandler.handle_arn of <botocore.utils.S3ArnParamHandler object at 0x7fd198488050>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.GetObject: calling handler <function generate_idempotent_uuid at 0x7fd1960b4b90>
DEBUG:botocore.hooks:Event before-c

time for dwave-qa: 
     local time is 25.150969743728638,
     task time is 16.808, 
     qpu total time is 0.020682, 
     qpu access time is 0.008576


In [7]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_process_result.save_mol_file(f"{timestamp}")

INFO:root:generate_optimize_pts()
DEBUG:root:generate_optimize_pts model_info={'model_name': '4_4_300_200', 'rb_name': ['30+31', '29+30', '28+29', '33+34'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4

KeyError: '12'

In [ ]:
qa_process_result.parameters

{'volume': {'optimize': 30.4620861938282,
  'initial': 32.03981237261626,
  'gain': 0.9507573215336145,
  'unfolding_results': ['x_15_2', 'x_14_2', 'x_12_3']}}